In [ ]:
import os

# latest spark version: 3.3.1
spark_version = 'spark-3.3.1'
os.environ['SPARK_VERSION']=spark_version

# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached findspark-2.0.1-py2.py3-none-any.whl (4.4 kB)


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-16 14:28:33--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.74MB/s    in 0.2s    

2022-11-16 14:28:34 (5.74 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles

# load in the file we wanna read
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

# create the DataFrame
vidgam_DF = spark.read.option('header','true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="yyyy/mm/dd")
vidgam_DF.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

* Complete the following steps for each notebook (one dataset per notebook).

  * Count the number of records (rows) in the dataset.

  * Transform the dataset to fit the tables in the [schema file](../Resources/schema.sql). Be sure that the DataFrames match in data type and in column name.

  * Load the DataFrames that correspond to tables into an RDS instance. **Note:** This process can take up to 10 minutes for each. Ensure that everything is correct before uploading.


In [ ]:
# number of records
print(f"Number of records in this dataset: {vidgam_DF.count()}")

Number of records in this dataset: 1785997


In [ ]:
# transform dataset to fit the tables

# print schema
vidgam_DF.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [ ]:
# Drop null values
vidgam_DF = vidgam_DF.dropna()

print(f"Number of rows after null values were removed: {vidgam_DF.count()}")

Number of rows after null values were removed: 1785886


In [ ]:
# change the datatypes
from pyspark.sql.types import TimestampType, IntegerType

vidgam_DF = vidgam_DF.withColumn("customer_id", vidgam_DF["customer_id"].cast(IntegerType()))\
.withColumn("product_parent", vidgam_DF["product_parent"].cast(IntegerType()))\
.withColumn("review_date", vidgam_DF["review_date"].cast(TimestampType()))\
.withColumn("star_rating", vidgam_DF["star_rating"].cast(IntegerType()))\
.withColumn("helpful_votes", vidgam_DF["helpful_votes"].cast(IntegerType()))\
.withColumn("total_votes", vidgam_DF["total_votes"].cast(IntegerType()))\

# Print schema
vidgam_DF.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# create review dataframe to match review_id_table table
review_id_DF = vidgam_DF.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_DF.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809|2015-08-31 00:00:00|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175|2015-08-31 00:00:00|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668|2015-08-31 00:00:00|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350|2015-08-31 00:00:00|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511|2015-08-31 00:00:00|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615|2015-08-31 00:00:00|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603|2015-08-31 00:00:00|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517|2015-08-31 00:00:00|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890|2015-08-31 00:00:00|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651|2015-08-31 00:00:00|
|R1VTIA3JTYBY02|     1065

In [ ]:
# Print schema
review_id_DF.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: timestamp (nullable = true)



In [ ]:
# create products dataframe to match products table(only unique values)
products_DF = vidgam_DF.select(["product_id","product_title"])
products_DF = products_DF.dropDuplicates()
products_DF.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# create customers dataframe to match customers table
from pyspark.sql.functions import desc

customers_DF = vidgam_DF.groupby("customer_id").agg({"customer_id":"count"})
customers_DF = customers_DF.orderBy(desc("count(customer_id)"))
customers_DF = customers_DF.withColumnRenamed("count(customer_id)", "customer_count") 
customers_DF.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52759271|           909|
|   42418272|           856|
|   45838740|           839|
|   24846840|           614|
|   24594001|           460|
|   40955995|           459|
|   48278407|           439|
|   27364030|           361|
|   30678701|           326|
|   50648042|           301|
|   14539589|           288|
|   10541465|           271|
|   38752775|           263|
|   43092063|           257|
|   45261621|           235|
|   14886512|           233|
|   44603910|           233|
|   37687227|           226|
|   10075230|           218|
|   52239651|           214|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# create vine dataframe to match vine_table 

# load in a sql function to use columns
from pyspark.sql.functions import col

vine_table_DF = vidgam_DF.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# filter for columns from Amazon's Vine program
vine_table_DF.show() 

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

In [ ]:
# configure RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://video-games-db.<removed>.us-east-1.rds.amazonaws.com:5432/video_game_DB"
config = {"user":"postgres",
          "password":"<removed>",
          "driver":"org.postgresql.Driver"}

In [ ]:
# write DataFrame to review_id_table in RDS

review_id_DF.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# write DataFrame to products_DF in RDS
products_DF.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# write DataFrame to customers_DF in RDS
customers_DF.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# write DataFrame to vine_table_DF in RDS
vine_table_DF.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)